# Import des librairies et des données

## Installation des librairies manquantes :
```

```

In [ ]:
import seaborn as sns
from matplotlib.colors import ListedColormap
import warnings

import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

import plotly.graph_objects as go
import pandas as pd
from sklearn import metrics
from sklearn.metrics import silhouette_score 
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

from scipy.spatial.distance import cdist
from scipy.cluster.hierarchy import dendrogram, linkage

from statsmodels.stats.outliers_influence import variance_inflation_factor

from yellowbrick.cluster import KElbowVisualizer

from Kmeans import KMeans as KMeans_custom

## Palette de couleurs

In [ ]:
palette = sns.color_palette("rocket")

darker = palette[0]
dark = palette[1]
medium = palette[2]
redish = palette[3]
light = palette[4]
lighter = palette[5]

colors = ['green' if (i == 0 and j == 0) or (i == 1 and j == 1) else 'red' for i in range(2) for j in range(2)]
cmap_cm = ListedColormap(colors)

sns.set_style('darkgrid')

cmap = sns.color_palette("rocket", as_cmap=True)

In [ ]:
df = pd.read_csv('CSV/marketing_campaign.csv', sep='\t')

# Analyse des données

In [ ]:
numberOfRowsBefore = df.shape[0]
print(f"Le dataset contient {df.shape[0]} lignes et {df.shape[1]} colonnes.")

In [ ]:
df.describe()

In [ ]:
df.drop('ID', axis=1, inplace=True)
df.drop('Z_CostContact', axis=1, inplace=True)
df.drop('Z_Revenue', axis=1, inplace=True)
df.drop_duplicates(inplace=True)
numberOfRowsAfter = df.shape[0]
df.describe()

In [ ]:
print(f"Nous avons supprimé {numberOfRowsBefore - numberOfRowsAfter} lignes en doublon.")

In [ ]:
def naValues(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum() / df.isnull().count()).sort_values(ascending=False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Pourcentage'])

naValues(df)

In [ ]:
df['Income'] = df['Income'].fillna(df['Income'].median())

In [ ]:
df['Marital_Status'].unique()

In [ ]:
df['Marital_Status'] = df['Marital_Status'].replace(['Alone', 'Absurd', 'YOLO', 'Divorced', 'Widow'], 'Single')
df['Marital_Status'] = df['Marital_Status'].replace(['Married', 'Together'], 'In couple')

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='Marital_Status', data=df, color=redish)
plt.title('Répartition des clients en fonction de leur statut marital')
plt.show()

In [ ]:
df['Outcome'] = df['MntFishProducts'] + df['MntMeatProducts'] + df['MntSweetProducts'] + df['MntFruits'] + df['MntWines'] + df['MntGoldProds']

df['Outcome'].head(10)

In [ ]:
df['Education'] = df['Education'].replace(['Basic', 'Graduation', '2n Cycle', 'Master', 'PhD'], [0, 1, 2, 3, 4])

In [ ]:
df['Education'].unique()

In [ ]:
df['TotalAcceptedCmp'] = df['AcceptedCmp1'] + df['AcceptedCmp2'] + df['AcceptedCmp3'] + df['AcceptedCmp4'] + df['AcceptedCmp5']

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='TotalAcceptedCmp', data=df, color=redish)
plt.title('Nombre de campagnes acceptées par les clients')
plt.show()

In [ ]:
df['TotalPurchases'] = df['NumWebPurchases'] + df['NumCatalogPurchases'] + df['NumStorePurchases'] + df['NumDealsPurchases']

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='TotalPurchases', data=df, color=redish)
plt.title('Nombre total d\'achats par les clients')
plt.show()

In [ ]:
df['Income'].plot(kind='hist', bins=50, color=dark, edgecolor='black', figsize=(10, 7))
plt.title('Histogramme du revenu')
plt.yscale('log')
plt.xlabel('Revenu')
plt.ylabel('Nombre de clients')
plt.show()

In [ ]:
dropCol = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']

df.drop(dropCol, axis=1, inplace=True)

<font color='blue'>**Observation :**</font>   
Je remarque qu'il y a quelques données abérrantes dans la colonne `Revenue`, je choisis donc de les supprimer

In [ ]:
df = df[df['Income'] <= 120000]
numberOfRowsAfter = df.shape[0]

df['Income'].plot(kind='hist', bins=50, color=dark, edgecolor='black', figsize=(10, 7))
plt.title('Histogramme du revenu')
plt.xlabel('Revenu')
plt.ylabel('Nombre de clients')
plt.show()

<font color='blue'>**Observation :**</font>   
On observe une bonne répartition du revenu des clients.

In [ ]:
df.describe()

In [ ]:
df['TotalChildHome'] = df['Kidhome'] + df['Teenhome']
plt.figure(figsize=(10, 7))
sns.scatterplot(x='TotalChildHome', y='Income', data=df, color=dark)
plt.title('Nombre d\'enfants total à charge par revenu')
plt.xlabel('Revenu')
plt.ylabel('Nombre d\'enfants total à charge')

plt.show()

<font color='blue'>**Observation :**</font>   
On remarque que les foyers les plus modestes sont en général plus nombreux que les foyers les plus aisés.

In [ ]:
df['Parents'] = df['Marital_Status'].apply(lambda x: 2 if x == 'In couple' else 1)

plt.figure(figsize=(10, 7))
sns.scatterplot(x='Parents', y='Income', data=df, color=dark)
plt.title('Nombre de parents par revenu')
plt.xlabel('Revenu')
plt.ylabel('Nombre de parents')

plt.show()

<font color='blue'>**Observation :**</font>   
On remarque une bonne répartition des clients par rapport à leurs status marital et leurs revenus.

<font color='blue'>**Observation :**</font>   
On remarque que la moyenne de revenu des clients diminue avec le nombre d'enfants (entre 0 et 2 enfants) puis augmente.

In [ ]:
plt.figure(figsize=(10, 7))
sns.heatmap(df.groupby(['TotalChildHome', 'Parents'])['Outcome'].mean().unstack(), cmap=cmap, annot=True, fmt=".0f")
plt.title('Dépense moyenne par nombre de parents et d\'enfants à charge')
plt.xlabel('Nombre de parents')
plt.ylabel('Nombre d\'enfants à charge')
plt.show()

<font color='blue'>**Observation :**</font>   
On constate que les dépenses diminuent quand le nombre d'enfants augmente.

In [ ]:
plt.figure(figsize=(19, 10))
plt.title('Matrice de corrélation')
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".2f", cbar_kws={'label': 'Coefficient de corrélation'})
plt.yticks(rotation=0)
plt.xticks(rotation=60)
plt.show()

<span style="color:blue">**Observation :**</span>   
Forte corrélation entre le revenu, les dépenses et le total d'achat.   
Forte corrélation négative entre le revenu et le nombre de visite.   
Corrélation moyenne entre le nombre de campagne promotionnelle, le revenu, les dépenses et la réponse à la dernière campagne.   
Corrélation moyenne positives entre le nombre d'enfants et le nombres de visites.   
Corrélation moyenne négative entre le nombre d'enfants, revenu et dépenses.   

<font color='red'>**Remarque :**</font>   
La variable cible (y) est le nombre de produit vendu par types de clients.

Le type de client (X) est défini par le revenu, le nombre d'enfants, le status marital, le nombre d'achat par moyens de communication.

In [ ]:
y = df[['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']]

X = df[['Year_Birth', 'Education', 'Income', 'Kidhome', 'Teenhome', 'Recency', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'Complain', 'Response', 'Outcome', 'TotalAcceptedCmp', 'TotalPurchases', 'TotalChildHome', 'Parents']]

# Réduction de dimension

## Sélection de features

Pour la sélection de features, il est possible d'utiliser plusieurs méthodes :

- **Zéro ou presque zéro variance** : On supprime les features qui ont une variance très faible.

- **Valeurs manquantes** : On supprime les features qui ont un pourcentage de valeurs manquantes très élevé.

- **Multicollinéarité** : On supprime les features qui sont fortement corrélées entre elles.

### Zéro ou presque zéro variance:

In [ ]:
sel = VarianceThreshold(threshold=0.05)

X_selection = sel.fit_transform(X)

print(f"Nombre de colonnes avant la sélection : {X.shape[1]}")
print(f"Nombre de colonnes après la sélection : {X_selection.shape[1]}")

for i in range(X.shape[1]):
    if i not in sel.get_support(indices=True):
        print(f"Colonne {X.columns[i]} supprimée")

New_X = X.iloc[:, sel.get_support(indices=True)]

### Multicolinéarité:

On considère que les variables sont fortement corrélées si le coefficient de corrélation est supérieur à 10.

In [ ]:
vif_scores = [variance_inflation_factor(New_X.values, i) for i in range(New_X.shape[1])]

print("VIF scores :")
for i, vif in enumerate(vif_scores):
    print(f"{New_X.columns[i]} : {vif}")

Suppression des variables Year_Birth et Income.

In [ ]:
New_X = df[['Education', 'Kidhome', 'Teenhome', 'Recency', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'Response', 'Outcome', 'TotalAcceptedCmp', 'TotalPurchases', 'TotalChildHome', 'Parents']]

vif_scores = [variance_inflation_factor(New_X.values, i) for i in range(New_X.shape[1])]

print("VIF scores :")
for i, vif in enumerate(vif_scores):
    print(f"{New_X.columns[i]} : {vif}")

Suppression des variables Kidhome et Teenhome.

In [ ]:
New_X = df[['Education', 'Recency', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'Response', 'Outcome', 'TotalAcceptedCmp', 'TotalChildHome', 'Parents']]

vif_scores = [variance_inflation_factor(New_X.values, i) for i in range(New_X.shape[1])]

print("VIF scores :")
for i, vif in enumerate(vif_scores):
    print(f"{New_X.columns[i]} : {vif}")

X final :

In [ ]:
X = df[['Education', 'Recency', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'Response', 'Outcome', 'TotalAcceptedCmp', 'TotalChildHome', 'Parents']]

## Analyse Factorielle Multiple

L'AFM est une méthode factorielle adaptée à l'étude des tableaux dans lesquels un ensemble d'individus est décris par un ensemble de variables (qualitatives et/ou quantitatives) structurées en groupes. Elle peut être vue comme une extension de:
- l'ACP (Analyse en Composantes Principales) pour les données quantitatives,
- l'ACM (Analyse des Correspondances Multiples) pour les données qualitatives,
- l'AFDM (Analyse Factorielle des Données Mixtes) pour les données des deux types.

Nous utilisons des données quantitatives, nous allons donc utiliser l'ACP.

Analyse en Composantes Principales (ACP) :

In [ ]:
PCA = PCA(n_components=3)
PCA.fit(X)
X_PCA = PCA.transform(X)
PCA_df = pd.DataFrame(data=X_PCA, columns=['PCA1', 'PCA2', 'PCA3'])

PCA_df.describe()

In [ ]:
fig = px.scatter_3d(PCA_df, x='PCA1', y='PCA2', z='PCA3', width=800, height=800, title='Projection 3D des composantes principales')
fig.show()

# Sélection du nombre de clusters

## Méthode du coude :

In [ ]:
Elbow_M = KElbowVisualizer(KMeans(), k=10)
Elbow_M.fit(PCA_df)
Elbow_M.title = 'Méthode du coude en fonction de la somme des carrés des distances et du temps d\'exécution'
Elbow_M.show()

## Méthode de Silhouette :

In [ ]:
silhouette_scores = []
K = range(2, 11)

for k in K:
    cluster = KMeans(n_clusters=k)
    cluster_labels = cluster.fit_predict(PCA_df)
    
    silhouette_scores.append(silhouette_score(PCA_df, cluster_labels))
    silhouette_average = silhouette_score(PCA_df, cluster_labels)

    print("Pour k clusters =", k, "La moyenne du score de silhouette est :", silhouette_average)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plt.plot(K, silhouette_scores)
plt.xlabel('Valeurs de K')
plt.ylabel('Score de silhouette')
plt.title('Score de silhouette en fonction de K')
plt.show()

On sélectionne 4 clusters.

# K-means

In [ ]:
kmeans = KMeans(n_clusters=4).fit(PCA_df)
y_kmeans = kmeans.predict(PCA_df)

fig = px.scatter_3d(PCA_df, x='PCA1', y='PCA2', z='PCA3', color=y_kmeans, width=800, height=800, title='Kmeans avec 4 clusters')
fig.show()

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(PCA_df['PCA1'], PCA_df['PCA2'], PCA_df['PCA3'], c=y_kmeans, cmap='viridis')
plt.title('Kmeans avec 4 clusters')
plt.show()

In [ ]:
silhouette_average_kmeans = silhouette_score(PCA_df, y_kmeans)
print("La moyenne du score de silhouette est :", silhouette_average_kmeans)

<font color='blue'>**Observation :**</font>

## Clustering Hiérarchique ascendant

In [ ]:
ward_cluster = AgglomerativeClustering(n_clusters=4, linkage='ward')
ward_labels = ward_cluster.fit_predict(PCA_df)

fig = px.scatter_3d(PCA_df, x='PCA1', y='PCA2', z='PCA3', color=ward_labels, width=800, height=800, title='Agglomerative Clustering avec 4 clusters')
fig.show()

### Dendrogramme

In [ ]:
dendrogram = dendrogram(linkage(PCA_df, method='ward'))
plt.title('Dendrogramme')
plt.gca().set_xticklabels([])
plt.show()

In [ ]:
silhouette_average_ach = silhouette_score(PCA_df, ward_labels)
print("La moyenne du score de silhouette est :", silhouette_average_ach)

<font color='blue'>**Observation :**</font>

On observe que la distance entre clusters est drastiquement réduite à partir de 4 clusters.

## Clustering DBSCAN

Détermination des hyperparamètres :

eps : distance maximale entre deux échantillons pour être considérés comme dans le même voisinage.

MinPts : nombre minimal de points requis pour former un cluster. Doit être choisi en fonction du nombre de dimension des données.

In [ ]:
MinPts = 4

In [ ]:
neighbors = NearestNeighbors(n_neighbors=MinPts)
neighbors_fit = neighbors.fit(PCA_df)
distances, indices = neighbors_fit.kneighbors(PCA_df)

In [ ]:
distances = np.sort(distances, axis=0)
distances = distances[:, 1]
plt.figure(figsize=(10, 6))
plt.title('Distance entre les points en fonction du nombre de voisins')
plt.plot(distances)

<font color='blue'>**Observation :**</font>

Epsilon est déterminé par le coude de la courbe de distance.   
Il est compris entre 10 et 15.
Nous choisissons 12.

In [ ]:
eps = 12

In [ ]:
Dbscan = DBSCAN(eps=eps, min_samples=MinPts)
DBSCAN_labels = Dbscan.fit_predict(PCA_df)

fig = px.scatter_3d(PCA_df, x='PCA1', y='PCA2', z='PCA3', color=DBSCAN_labels, width=800, height=800, title='DBSCAN avec eps=12 et MinPts=4')
fig.show()

In [ ]:
silhouette_average_Dbscan = silhouette_score(PCA_df, DBSCAN_labels)
print("La moyenne du score de silhouette est :", silhouette_average_Dbscan)

<font color='red'>**Remarque :**</font>
L'algorithme DBSCAN ne permet pas de séparer les données en 4 clusters.
En effet, les données sont trop rapprochées pour pouvoir être séparées.

## Comparaison des résultats

In [ ]:
print("Le score de silhouette de Kmeans est :", silhouette_average_kmeans.round(2))
print("Le score de silhouette de Agglomerative Clustering est :", silhouette_average_ach.round(2))
print("Le score de silhouette de DBSCAN est :", silhouette_average_Dbscan.round(2))

Le clustering hiérarchique ascendant et le K-means donnent des scores de silhouette quasiment similaires (63% - 61%) tandis que le DBSCAN donne un score beaucoup plus faible (8%).

Ce qui peut être expliqué par le fait que le DBSCAN se base sur la densité des données pour les regrouper, or les données sont trop rapprochées pour être séparées.

# Profilage de la clientèle

<font color='red'>**TODO :**</font>
Prendre ses clusters, et plotter par rapport aux différentes variables afin de voir les différences entre les clusters.

In [ ]:
plt.title('y_kmeans par éducation')
sns.boxplot(x=y_kmeans, y=df['Education'], palette='rocket')
plt.show()

In [ ]:
plt.title('y_kmeans par kidhome')
sns.boxplot(x=y_kmeans, y=df['Kidhome'], palette='rocket')
plt.show()

In [ ]:
plt.title('y_kmeans par teenhome')
sns.boxplot(x=y_kmeans, y=df['Teenhome'], palette='rocket')
plt.show()

In [ ]:
plt.title('y_kmeans par totalchildhome')
sns.boxplot(x=y_kmeans, y=df['TotalChildHome'], palette='rocket')
plt.show()

In [ ]:
plt.title('y_kmeans par income')
sns.boxplot(x=y_kmeans, y=df['Income'], palette='rocket')
plt.show()

In [ ]:
plt.title('y_kmeans par outcome')
sns.boxplot(x=y_kmeans, y=df['Outcome'], palette='rocket')
plt.show()

Cluster 0: Plus nombreux (enfant + adolescent), revenus plus faible, dépense très faible.

Cluster 1: Quasiment pas d'enfant, mais 1 ou 2 adolescents, revenue moyen - haut, dépense moyenne.

Cluster 2: globalement 1 enfant ou 1 adolescent, revenu moyen, dépense moyenne - faible.

Cluster 3: généralement pas d'enfant, revenu moyen - haut, dépense moyenne - hautte